In [1]:
import os
# Find the latest version of spark 3.0 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.0.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:3 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:4 http://security.ubuntu.com/ubuntu bionic-security InRelease
Hit:5 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:6 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:7 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Hit:8 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Ign:9 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:10 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:11 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:13 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic 

In [2]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2021-11-24 17:24:54--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar.1’

postgresql-42.2.16. 100%[===================>] 979.38K  --.-KB/s    in 0.1s    

2021-11-24 17:24:55 (6.55 MB/s) - ‘postgresql-42.2.16.jar.1’ saved [1002883/1002883]



In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("M16-Amazon-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [4]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Home_Improvement_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get(""), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   48881148|R215C9BDXTDQOW|B00FR4YQYK|     381800308|SadoTech Model C ...|Home Improvement|          4|            0|          0|   N|                Y|          Four Stars|        good product| 2015-08-31|
|         US|   47882936|R1DTPUV1J57YHA|B00439MYYE|     921341748|iSpring T32M 3.2 ...|Home Improvement|          5|    

In [5]:
totalVotes_df = df.filter('total_votes >= 20')
totalVotes_df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   42943632|R2VIQ3UX794Q0O|B00YMNSCYM|      74738477|LE LED Under Cabi...|Home Improvement|          4|           21|         21|   N|                Y|Nice Kit, I showe...|Nice Kit, I showe...| 2015-08-31|
|         US|   24649241|R1OA24IIHWF54G|B00YI5TNXI|     813068315|Hyperikon BR20-Se...|Home Improvement|          5|    

In [6]:
helpfulVotes_df = totalVotes_df.filter('helpful_votes / total_votes >= 50/100')
helpfulVotes_df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   42943632|R2VIQ3UX794Q0O|B00YMNSCYM|      74738477|LE LED Under Cabi...|Home Improvement|          4|           21|         21|   N|                Y|Nice Kit, I showe...|Nice Kit, I showe...| 2015-08-31|
|         US|   24649241|R1OA24IIHWF54G|B00YI5TNXI|     813068315|Hyperikon BR20-Se...|Home Improvement|          5|    

In [7]:
vine_df = helpfulVotes_df.filter(helpfulVotes_df['vine']=='Y')
vine_df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   53093730|R29V4UDSS053D8|B00P6ORMR4|     406381545|Aquasana AQ-RO-3....|Home Improvement|          3|           33|         38|   Y|                N|Not for average h...|This filter is ov...| 2015-08-30|
|         US|   25984637|R1I2D068WC37PA|B00ZWBURWI|     302288140|Myfox Security Ca...|Home Improvement|          3|    

In [8]:
nonVine_df = helpfulVotes_df.filter(helpfulVotes_df['vine']=='N')
nonVine_df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   42943632|R2VIQ3UX794Q0O|B00YMNSCYM|      74738477|LE LED Under Cabi...|Home Improvement|          4|           21|         21|   N|                Y|Nice Kit, I showe...|Nice Kit, I showe...| 2015-08-31|
|         US|   24649241|R1OA24IIHWF54G|B00YI5TNXI|     813068315|Hyperikon BR20-Se...|Home Improvement|          5|    

In [18]:
vine_reviews = vine_df.count()
print("Total number of vine reviews are: %d" % vine_reviews)

Total number of paid vine reviews are: 266


In [19]:
five_star_vineReviews = vine_df.filter(vine_df.star_rating == 5).count()
print("The total number of five star vine reviews are: %d" % five_star_vineReviews)

The total number of five star paid vine reviews are: 125


In [20]:
five_star_vineReviews_percentage = (five_star_vineReviews/vine_reviews)*100
print("The five star vine review percentage is: %.2f" % five_star_vineReviews_percentage) 

The five star paid vine review percentage is: 46.99


In [21]:
nonVine_review = nonVine_df.count()
print(" The total number of non vine reviews are: %d" % nonVine_review)

 The total number of non paid vine reviews are: 38829


In [22]:
five_star_nonVineReviews = nonVine_df.filter(nonVine_df.star_rating == 5).count()
print(" The five star non vine reviews are: %d" % five_star_nonVineReviews)

 The five star non vine reviews are: 18246


In [23]:
five_star_nonVineReviews_percentage = (five_star_nonVineReviews/nonVine_review) * 100
print(" The percentage of five star non vine reviews is: %.2f" % five_star_nonVineReviews_percentage)

 The percentage of five star non vine reviews is: 46.99
